#0. LIBRARIES

In [32]:
import sqlite3
import pandas as pd
import os
import google.generativeai as genai

## Pre-congifs

In [ ]:
# ---- CONFIG ----
CSV_FILE = "crime_reports.csv"
DB_FILE = "crime_reports.db"

# ---- GET GEMINI API FROM ENV ----
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GEMINI_API_KEY:
    raise Exception("Please set GEMINI_API_KEY in environment variables.")

#1. LOAD CSV

In [28]:
df = pd.read_csv(CSV_FILE)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2981 entries, 0 to 2980
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   report_id            2981 non-null   int64  
 1   date_reported        2981 non-null   object 
 2   date_occurred        2981 non-null   object 
 3   time_occurred        2981 non-null   int64  
 4   area_code            2981 non-null   int64  
 5   area_name            2981 non-null   object 
 6   reporting_district   2981 non-null   int64  
 7   crime_part           2981 non-null   int64  
 8   crime_code           2981 non-null   int64  
 9   crime_description    2981 non-null   object 
 10  modus_operandi       2535 non-null   object 
 11  victim_age           2981 non-null   int64  
 12  victim_sex           2554 non-null   object 
 13  victim_descent       2554 non-null   object 
 14  premise_code         2981 non-null   int64  
 15  premise_description  2979 non-null   o

In [29]:
df.head()

,report_id,date_reported,date_occurred,time_occurred,area_code,area_name,reporting_district,crime_part,crime_code,crime_description,...,status_code,status_description,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,190326475,2020-03-01,2020-03-01,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,2020-02-09,2020-02-08,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,2020-11-11,2020-11-04,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,2023-05-10,2020-03-10,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,2020-09-09,2020-09-09,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


# 2. CREATE SQLITE TABLE DYNAMICALLY

In [30]:
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

In [31]:
cursor

In [33]:
def generate_schema(df):
    schema = []
    for column in df.columns:
        dtype = df[column].dtype
        if dtype == 'int64':
            schema.append(f"{column} INTEGER")
        elif dtype == 'float64':
            schema.append(f"{column} REAL")
        elif dtype == 'bool':
            schema.append(f"{column} BOOLEAN")
        elif dtype == 'object':
            schema.append(f"{column} TEXT")
        elif dtype == 'datetime64[ns]':
            schema.append(f"{column} DATE")
        else:
            schema.append(f"{column} TEXT")

    return f"CREATE TABLE IF NOT EXISTS crime_reports (\n  {', '.join(schema)}\n);"

create_table_sql = generate_schema(df)
cursor.execute(create_table_sql)
conn.commit()

In [34]:
def insert_data(df, conn):
    cols = df.columns.tolist()
    placeholders = ', '.join(['?' for _ in cols])
    insert_sql = f"INSERT INTO crime_reports ({', '.join(cols)}) VALUES ({placeholders})"
    for _, row in df.iterrows():
        cursor.execute(insert_sql, tuple(row))
    conn.commit()

insert_data(df, conn)

In [35]:
# ---- 3. GENERATE THINGMAKER METADATA ----
def thingmaker(df):
    info = []
    for col in df.columns:
        dtype = df[col].dtype
        col_info = f"{col} - "
        if dtype == 'int64' or dtype == 'float64':
            col_info += f"int - ({df[col].min()} - {df[col].max()})"
        elif dtype == 'bool':
            col_info += "bool - (True, False)"
        elif dtype == 'object':
            uniq = df[col].nunique()
            if uniq < 15:
                col_info += f"string - {list(df[col].dropna().unique())}"
            else:
                col_info += "string - NA"
        elif dtype == 'datetime64[ns]':
            col_info += f"date - ({df[col].min().strftime('%Y-%m-%d')} - {df[col].max().strftime('%Y-%m-%d')})"
        else:
            col_info += "unknown - NA"
        info.append(col_info)
    return "\n".join(info)

thingmaker_metadata = thingmaker(df)
print("\n---- THINGMAKER METADATA ----\n")
print(thingmaker_metadata)


---- THINGMAKER METADATA ----

report_id - int - (190326475 - 201309918)
date_reported - string - NA
date_occurred - string - NA
time_occurred - int - (1 - 2355)
area_code - int - (1 - 13)
area_name - string - ['Wilshire', 'Central', 'Southwest', 'Van Nuys', 'Hollenbeck', 'Rampart', 'Newton', 'Northeast', '77th Street', 'Hollywood', 'Harbor', 'West Valley', 'West LA']
reporting_district - int - (101 - 1395)
crime_part - int - (1 - 2)
crime_code - int - (110 - 956)
crime_description - string - NA
modus_operandi - string - NA
victim_age - int - (0 - 99)
victim_sex - string - ['M', 'X', 'F']
victim_descent - string - ['O', 'X', 'W', 'H', 'B', 'A', 'K', 'C', 'F', 'Z', 'V', 'I', 'P', 'J']
premise_code - int - (101 - 966)
premise_description - string - NA
weapon_code - int - (101.0 - 515.0)
weapon_description - string - NA
status_code - string - ['AA', 'IC', 'AO', 'JA', 'JO']
status_description - string - ['Adult Arrest', 'Invest Cont', 'Adult Other', 'Juv Arrest', 'Juv Other']
crime_code_1

In [ ]:
# ---- 4. CONFIGURE GEMINI ----
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

In [37]:
model

genai.GenerativeModel(
    model_name='models/gemini-2.0-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [41]:
# ---- 5. MAGICPIPELINE ----
def clean_sql_response(text):
    """Remove ```sql ``` or ``` code fences from Gemini output"""
    text = text.strip()
    if text.startswith("```"):
        text = text.split('```')[-1]
    return text.strip()

def magicpipeline(user_question, thingmaker_metadata):
    # ---- Generate SQL ----
    prompt_sql = f"""
You are an AI that converts user questions into valid SQL for SQLite.
Schema info:
table name {CSV_FILE.split('.')[0]}
{thingmaker_metadata}
Write an SQL query for:
"{user_question}"
Only return the SQL query. Do NOT use ```sql or ``` code fences.
"""
    sql_response = model.generate_content(prompt_sql)
    sql_query = clean_sql_response(sql_response.text)

    print(f"\nGenerated SQL: {sql_query}")

    # ---- Execute SQL ----
    try:
        conn = sqlite3.connect(DB_FILE)
        df_result = pd.read_sql_query(sql_query, conn)
        conn.close()
    except Exception as e:
        return f"SQL execution error: {e}"

    # ---- Generate Final Answer ----
    result_markdown = df_result.to_markdown(index=False)
    prompt_answer = f"""
User question: "{user_question}"
SQL query result:
{result_markdown}
Write a clear answer:
"""
    answer_response = model.generate_content(prompt_answer)
    return answer_response.text.strip()



In [43]:

# ---- 6. INTERACTIVE LOOP ----
while True:
    user_q = input("\nAsk a question (type 'exit' to quit): ")
    if user_q.lower() == 'exit':
        print("Goodbye!")
        break
    answer = magicpipeline(user_q, thingmaker_metadata)
    print(f"\nAnswer:\n{answer}")


Ask a question (type 'exit' to quit): hello

Generated SQL: SELECT * FROM crime_reports LIMIT 1;

Answer:
Hello! How can I help you with the provided crime data? Do you have any specific questions you want to ask or analyses you'd like me to perform? For example, you could ask me things like:

*   "How many vehicle thefts occurred in the Wilshire area in March 2020?"
*   "What are the most common crime descriptions?"
*   "What are the different status descriptions in the dataset?"
*   "What are the locations where crimes were reported?"
*   "Find all crimes committed using a weapon."

Ask a question (type 'exit' to quit): how many crimes are in city wilshire

Generated SQL: SELECT count(*) FROM crime_reports WHERE area_name = "Wilshire"

Answer:
There are 492 crimes in Wilshire.

Ask a question (type 'exit' to quit): how many of those crime in wilshire are cmmied by men

Generated SQL: SELECT count(*) FROM crime_reports WHERE area_name = 'Wilshire' AND victim_sex = 'M'

Answer:
Based 